In [56]:
from pynq import Overlay, MMIO
from time import sleep
from datetime import datetime

class NEC_IR_Controller:
    def __init__(self, bitfile_path):
        # Load the overlay
        self.ol = Overlay(bitfile_path)
        self.ol.download()
        
        # Configure GPIO addresses (adjust these to match your Vivado design)
        self.NEC_GPIO_BASE = 0x41200000  # Output for NEC transmission
        
        # Initialize MMIO interfaces
        self.nec_gpio = MMIO(self.NEC_GPIO_BASE, 0x10000)
        
        # Configure directions:
        self.nec_gpio.write(0x04, 0x00000000)  # All outputs for NEC
        self.nec_gpio.write(0x0C, 0x00000000)  # All outputs for send control
        
        # Timing parameters
        self.NEC_HOLD_TIME = 0.02  # 20ms hold for NEC frame
        self.BTN_PULSE_WIDTH = 0.001  # 1ms pulse width for button

    def send_nec_command(self, value):
        """Send a 32-bit NEC command to the PL transmitter with button trigger"""
        # Step 1: Write the NEC command
        self.nec_gpio.write(0x00, 0)
        sleep(self.BTN_PULSE_WIDTH)
        self.nec_gpio.write(0x00, value)
        
        # Step 2: Pulse the button signal
        self.nec_gpio.write(0x08, 0x1)  # Assert button
        sleep(self.BTN_PULSE_WIDTH)
        self.nec_gpio.write(0x08, 0x0)  # Deassert button
        
        # Step 3: Hold the NEC command for the required duration
        sleep(self.NEC_HOLD_TIME)
        
        # Step 4: Clear the NEC command
        self.nec_gpio.write(0x00, 0)
    def send_user_command(self,cmd):
        if(cmd == '-'):
            self.send_nec_command(0xE0)
            return;
        if(cmd == '+'):
            self.send_nec_command(0xA8)
            return;
        if(cmd == '0'):
            self.send_nec_command(0x68)
            return;
        if(cmd == '1'):
            self.send_nec_command(0x30)
            return;
        if(cmd == '2'):
            self.send_nec_command(0x18)
            return;
        if(cmd == '3'):
            self.send_nec_command(0x7A)
            return;
        if(cmd == '4'):
            self.send_nec_command(0x10)
            return;
        if(cmd == '5'):
            self.send_nec_command(0x38)
            return;
        if(cmd == '6'):
            self.send_nec_command(0x5A)
            return;
        if(cmd == '7'):
            self.send_nec_command(0x42)
            return;
        if(cmd == '8'):
            self.send_nec_command(0x4A)
            return;
        if(cmd == '9'):
            self.send_nec_command(0x52)
            return;
        else:
            self.send_nec_command(0x00)
            return;
            
    def send_full_command(self,str_cmd):
        for i in str_cmd:
            self.send_user_command(i)
            sleep(0.025) #20ms sleep time  
        
        

# Example usage:
if __name__ == "__main__":
    # Initialize with correct class name
    ir = NEC_IR_Controller(
        "/home/xilinx/jupyter_notebooks/xilinx/overlays/own/design_1_wrapper.bit"
    )
    
    try:
        a = datetime.now()
        for i in range(100):
            ir.send_full_command('-0005+')
        print(datetime.now() - a)#estimating the time of i code sending
            
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Clean up GPIO states
        ir.nec_gpio.write(0x00, 0)
        ir.nec_gpio.write(0x08, 0)
        print("Cleanup complete")

0:00:25.531279
Cleanup complete
